In [7]:
from IPython.display import display
import time
import pandas as pd
import os
import shutil
from sqlalchemy import *
from urllib.parse import quote_plus
import numpy as np
import sys
import traceback
from stock_processed_sql import *
from ETL_pipeline import *
from infrastructure import *
import requests
import json
import random


In [2]:
# Database connection setup
db_user = 'tkan'
db_password = 'Maihainganha@1'
db_host = 'localhost'  
db_port = '5432'       
db_name = 'finance_db' 
password = quote_plus(db_password) # Encode password
connection_str = f'postgresql://{db_user}:{password}@{db_host}:{db_port}/{db_name}' # Connection string
engine = create_engine(connection_str) # Create engine
inspector = inspect(engine) # Create inspector
df_tickers = pd.read_sql('SELECT "Ticker" FROM analysis_data.companies_list', engine)
ticker_list = set(df_tickers['Ticker'].str.strip())

In [13]:
quarter_tickers = set()
df_tickers = pd.read_sql('SELECT "Ticker" FROM analysis_data.companies_list', engine)
ticker_list = set(df_tickers['Ticker'].str.strip())
if inspector.has_table('ic_quarter', schema='raw'):
    df_all_quarter_ic = pd.read_sql('SELECT "Ticker" FROM raw.ic_quarter', engine)
    quarter_tickers = set(df_all_quarter_ic['Ticker'].str.strip())
missing_tickers = set(ticker_list - quarter_tickers)
print(len(missing_tickers))

for i,ticker in enumerate(missing_tickers):
    try:
        print(f'Processing {i+1}/{len(missing_tickers)}: {ticker}')
        co_phieu = ticker
        start = 2020
        end = 2025
        years = range(end, start-1, -1)
        quarters = [4,3,2,1]
        auth_token = 'Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoxODg5NjIyNTMwLCJuYmYiOjE1ODk2MjI1MzAsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsiYWNhZGVteS1yZWFkIiwiYWNhZGVteS13cml0ZSIsImFjY291bnRzLXJlYWQiLCJhY2NvdW50cy13cml0ZSIsImJsb2ctcmVhZCIsImNvbXBhbmllcy1yZWFkIiwiZmluYW5jZS1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImludmVzdG9wZWRpYS1yZWFkIiwib3JkZXJzLXJlYWQiLCJvcmRlcnMtd3JpdGUiLCJwb3N0cy1yZWFkIiwicG9zdHMtd3JpdGUiLCJzZWFyY2giLCJzeW1ib2xzLXJlYWQiLCJ1c2VyLWRhdGEtcmVhZCIsInVzZXItZGF0YS13cml0ZSIsInVzZXJzLXJlYWQiXSwianRpIjoiMjYxYTZhYWQ2MTQ5Njk1ZmJiYzcwODM5MjM0Njc1NWQifQ.dA5-HVzWv-BRfEiAd24uNBiBxASO-PAyWeWESovZm_hj4aXMAZA1-bWNZeXt88dqogo18AwpDQ-h6gefLPdZSFrG5umC1dVWaeYvUnGm62g4XS29fj6p01dhKNNqrsu5KrhnhdnKYVv9VdmbmqDfWR8wDgglk5cJFqalzq6dJWJInFQEPmUs9BW_Zs8tQDn-i5r4tYq2U8vCdqptXoM7YgPllXaPVDeccC9QNu2Xlp9WUvoROzoQXg25lFub1IYkTrM66gJ6t9fJRZToewCt495WNEOQFa_rwLCZ1QwzvL0iYkONHS_jZ0BOhBCdW9dWSawD6iF1SIQaFROvMDH1rg'
        user = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        headers = { 'Authorization': auth_token,
                    'User-Agent': user}
        query = text("""
                    INSERT INTO raw.ic_quarter ("Ticker", "Year", "Quarter", "data")
                    VALUES (:ticker, :year, :quarter, :data)
                    ON CONFLICT ("Ticker", "Year", "Quarter") 
                    DO UPDATE SET 
                    "data" = EXCLUDED."data",
                    "insert at" = CURRENT_TIMESTAMP;
                    
        """)
        with engine.connect() as conn:
            transaction = conn.begin()
            if inspector.has_table('ic_quarter', schema='raw'):
                try:
                    for y in years:
                        print(f'Year: {y}')
                        for q in quarters:
                            print(f'  Quarter: {q}')
                            url = f'https://restv2.fireant.vn/symbols/{co_phieu}/full-financial-reports?type=2&year={y}&quarter={q}&limit=1'
                            params = { 'type': '2', 
                            'year': str(y), 
                            'quarter': str(q), 
                            'limit': '1'}
                            response = requests.get(url, headers=headers, params=params)
                            if response.status_code == 200:
                                data = response.json()
                                if not data:
                                    continue
                                resp_json = json.dumps(data, ensure_ascii=False)
                                conn.execute(query, {"ticker": co_phieu, "year": y, "quarter": q, "data": resp_json})
                            time.sleep(1) # Thời gian chờ giữa các yêu cầu để tránh bị chặn
                    print(f'xử lý xong cho {co_phieu}')
                    transaction.commit()
                except Exception as e:
                    transaction.rollback()
                    print(f"Error: {e}")
                    traceback.print_exc()
    except Exception as e:
        print(f"Failed to process {ticker}: {e}")
        traceback.print_exc()



  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2022
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2021
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2020
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
xử lý xong cho CLH
Processing 475/638: VPI
Year: 2025
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2024
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2023
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2022
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2021
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2020
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
xử lý xong cho VPI
Processing 476/638: SCR
Year: 2025
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2024
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2023
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2022
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quarter: 1
Year: 2021
  Quarter: 4
  Quarter: 3
  Quarter: 2
  Quar